In [1]:
!pip install googledrivedownloader
!pip install keras

In [2]:
from google_drive_downloader import GoogleDriveDownloader as gdd

ids =["1m3o5jRXxaE19iCv2s0LKIGZppJrKk0zX", "1QGb2o6ISDtaAS6eK2bYmFsdK_xHSbYNX","19KQfcz7GTArRCPsk61IVrAqLkUlk31Yn",
      "16GA5jJYJ7qaypO5OdbesKd_19ObDS4AQ", "1tEGfozl4bfjzNAMZDWjOAx7yCzhBSfVH"]

file_names = ["declarative_question.pkl","description_statement.pkl", "view_opinion_statement.pkl",
              "wh_question.pkl", "yes_no_question.pkl"]

for i in range(len(ids)):
  a = gdd.download_file_from_google_drive(file_id= ids[i],
                                      dest_path="./"+file_names[i],
                                      unzip=False)

In [0]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten, BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras import backend as K
from keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
K.set_image_dim_ordering('th')
import pickle

In [0]:
"""words list represents the "pkl" file names
    which have the same word as .wav file.
    The pkl file is dictionary with key as the word spoken
    and the value as numpy array where element is a (20 * 100) vector
    representing the mfcc vectors of the .wav file. """

words = ["declarative_question", "description_statement", "view_opinion_statement", "wh_question", "yes_no_question"]
#words = ["bed", "down", "dog"]

In [14]:
data = pickle.load( open( words[0]+".pkl", "rb" ) )
for keys in data:
    X = data[keys]
    Y = np.array([[keys]] * data[keys].shape[0])
    #print X_1.shape
    
    
for i in range(1,len(words)):
    data = pickle.load( open( words[i]+".pkl", "rb" ) )
    for keys in data:
        Y = np.append(Y, [[keys]] * data[keys].shape[0], axis = 0)
        X = np.append(X, data[keys], axis = 0)
        #print X_1.shape
        
print X.shape
print Y.shape

(88, 20, 300)
(88, 1)


In [0]:
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(Y)
#print integer_encoded
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
Y = onehot_encoded

In [41]:
"""Convolution Layer in Keras
   accepts input as 3d - height, width, depth(RGB channels).
   Since we do not have depth in text, we can convert it to a vector
   with a depth = 1"""

X = X.reshape(X.shape[0], 1, 20, 300)
print X.shape
print Y.shape
num_classes = Y.shape[1]
print num_classes

(88, 1, 20, 300)
(88, 5)
5


In [0]:
model = Sequential()
model.add(Conv2D(50, (5, 5), input_shape=(1, 20, 300), activation='relu', padding="same"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(30, (3, 3), activation='relu', padding="same"))
model.add(BatchNormalization())
#model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(15, (3, 3), activation='relu', padding="same"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))


model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [56]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 50, 20, 300)       1300      
_________________________________________________________________
batch_normalization_13 (Batc (None, 50, 20, 300)       1200      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 50, 10, 150)       0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 30, 10, 150)       13530     
_________________________________________________________________
batch_normalization_14 (Batc (None, 30, 10, 150)       600       
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 15, 10, 150)       4065      
_________________________________________________________________
batch_normalization_15 (Batc (None, 15, 10, 150)       600       
__________

In [44]:
# file_path="CNN_v4_val_loss.hdf5"
# checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
# early = EarlyStopping(monitor="val_loss", mode="min", patience=20)
# callbacks_list = [checkpoint, early] #early
model.fit(X, Y, epochs=30, batch_size= 32)

Epoch 1/30
88/88 [==============================] - 1s 10ms/step - loss: 2.3007 - acc: 0.2500
Epoch 2/30
88/88 [==============================] - 0s 1ms/step - loss: 1.7944 - acc: 0.2727
Epoch 3/30
88/88 [==============================] - 0s 1ms/step - loss: 1.6859 - acc: 0.3409
Epoch 4/30
88/88 [==============================] - 0s 1ms/step - loss: 1.4641 - acc: 0.3295
Epoch 5/30
88/88 [==============================] - 0s 1ms/step - loss: 1.4620 - acc: 0.3409
Epoch 6/30
88/88 [==============================] - 0s 1ms/step - loss: 1.3053 - acc: 0.4545
Epoch 7/30
88/88 [==============================] - 0s 1ms/step - loss: 1.4274 - acc: 0.3864
Epoch 8/30
88/88 [==============================] - 0s 1ms/step - loss: 1.3838 - acc: 0.3864
Epoch 9/30
88/88 [==============================] - 0s 1ms/step - loss: 1.2569 - acc: 0.4659
Epoch 10/30
88/88 [==============================] - 0s 1ms/step - loss: 1.1743 - acc: 0.4886
Epoch 11/30
88/88 [==============================] - 0s 1ms/step - l

88/88 [==============================] - 0s 1ms/step - loss: 0.0845 - acc: 0.9773
Epoch 30/30
88/88 [==============================] - 0s 1ms/step - loss: 0.0466 - acc: 1.0000


In [0]:
predicted = model.predict(X)

In [0]:
from sklearn import metrics

In [47]:
metrics.log_loss(Y, predicted)

0.6807367339814928

In [0]:
model.save('dialog_speech_CNN.h5')

In [0]:
model = load_model('dialog_speech_CNN.h5')

In [0]:
from google.colab import files

In [53]:
ls

datalab/                   dialog_speech_CNN.h5        yes_no_question.pkl
declarative_question.pkl   view_opinion_statement.pkl
description_statement.pkl  wh_question.pkl


In [0]:
files.download("dialog_speech_CNN.h5")